<a href="https://colab.research.google.com/github/zamalex/BakingFinal/blob/master/lightweight%20HART%20base%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import sklearn.metrics
from tensorflow.keras.utils import plot_model
import datetime


In [2]:
%load_ext tensorboard


# Some methods and data which will be used below

In [3]:
# (i).   Mapping labels to their resp. classes
# (ii).  Mapping classes to their resp. labels

label_to_class = {
    1  : 'WALKING',
    2  : 'WALKING_UPSTAIRS',
    3  : 'WALKING_DOWNSTAIRS',
    4  : 'SITTING',
    5  : 'STANDING',
    6  : 'LAYING',
    7  : 'STAND_TO_SIT',
    8  : 'SIT_TO_STAND',
    9  : 'SIT_TO_LIE',
    10 : 'LIE_TO_SIT',
    11 : 'STAND_TO_LIE',
    12 : 'LIE_TO_STAND',
    np.nan : np.nan
}
class_to_label = {
    'WALKING' : 1,
    'WALKING_UPSTAIRS' : 2,
    'WALKING_DOWNSTAIRS' : 3,
    'SITTING' : 4,
    'STANDING' : 5,
    'LAYING' : 6,
    'STAND_TO_SIT' : 7,
    'SIT_TO_STAND' : 8,
    'SIT_TO_LIE' : 9,
    'LIE_TO_SIT' : 10,
    'STAND_TO_LIE' : 11,
    'LIE_TO_STAND' : 12,
    np.nan : np.nan
}

In [4]:
# (i).   Removing data-points where y and x values is null

# Other methods can be
# ffill (forward fill) => fills using forward points
# bfill (backward fill) => using backward points
# interpolate

def remove_null(xdata,ydata):
    xdata = xdata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(ydata))]
    ydata = ydata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]
    xdata = xdata[np.where(np.isfinite(xdata).all(axis = 1).all(axis = 1).all(axis = 1))]

    return xdata,ydata

In [5]:
# normalize xdata using sklearn.preprocessing.StandardScaler and returns
# scaler object to use it furthur for testing data

# Each axis of each sensor has different min, max, I scaled according to them seperately
# Initial shape == (None,128,2,3)
# changed to (None , 6) :-
# reshape to (None,128,6) -> swapaxis(0,2) -> reshape(6,-1) -> transpose
# Fit scaler OR transform according to scaler

# Reverse above process to get back oiginal data
# transpose -> reshape(6,128,None) -> swapaxes(0,2) -> reshape(None,128,2,3)

def get_scaler(xdata):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    scaler = MinMaxScaler(feature_range = (-1,1))
    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    scaler.fit(xdata)
    return scaler

def scale_data(xdata,scaler):

    row = xdata.shape[0]
    timestamp = xdata.shape[1]
    sensor = xdata.shape[2]
    axis = xdata.shape[3]

    xdata = xdata.reshape(row,timestamp,sensor*axis)
    xdata = np.swapaxes(xdata,0,2).reshape(sensor*axis,-1).T
    xdata = scaler.transform(xdata)
    xdata = xdata.T.reshape(sensor*axis,timestamp,row)
    xdata = np.swapaxes(xdata,0,2).reshape(row,timestamp,sensor,axis)

    return xdata

In [6]:
# takes in location, exp no., user no., start and end(end point is excluded from reading i.e lastpoint+1) point
# ,overlap array, and returns xdata and ydata

def create_windows(location, exp, user, start, end, activity, length, overlap):

    acc_file  = location + '/acc_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'
    gyro_file = location + '/gyro_exp'+ str(exp).zfill(2) + '_user' + str(user).zfill(2) + '.txt'

    acc_data  = np.loadtxt(acc_file)
    gyro_data = np.loadtxt(gyro_file)

    xtrain = []
    ytrain = []

    while (start + length <= end) :

        stop = start + length
        window = []

        while start != stop :
            window.append( [acc_data[start] , gyro_data[start]] )
            start += 1

        xtrain.append(window)
        ytrain.append(activity)

        start = stop - overlap[activity-1]

    return xtrain,ytrain

In [7]:
# location == location of file
# lenght == lenght of window
# overlap == array of overlaps of size == number of unique activities
# overlap depends on activity so as to extract more data from a particular class if needed


# (i).   Loading labels.txt as labels
# (ii).  Iterating in labels and calling create_windows on acceleration file, extending returned data in xtrain, ytrain
# (iii). Iterating in labels and calling create_windows on gyroscope file, extending returned data in xtrain, ytrain

def prepare_data(location,length = 128,overlap = [64]*12):

    xdata = []
    ydata = []

    labels = np.loadtxt(location+'/labels.txt',dtype = 'uint32')

    for exp,user,activity,start,end in labels :

        xtemp , ytemp = create_windows(location, exp, user, start, end+1, activity, length, overlap)
        xdata.extend(xtemp)
        ydata.extend(ytemp)

    return np.array(xdata),np.array(ydata)

In [8]:
# (i). Finds max element index sets its 1 and sets remaining 0
#      for each row

def to_categorical(ydata):

    for i in range(len(ydata)):
        j = ydata[i].argmax()
        for k in range(len(ydata[i])):
            ydata[i][k] = (k == j)
    return ydata

In [9]:
# (i).  OneHotEncoding ydata
# (ii). Converting sparsh matrix ydata into dense form and then matrix into numpy array

def one_hot_encoded(ydata):
    ydata = OneHotEncoder().fit_transform(ydata.reshape(len(ydata),1))
    ydata = np.asarray(ydata.todense())
    return ydata

def splitData(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)
  testlabels = ytest
  scaler = get_scaler(xtrain)
  xtrain = scale_data(xtrain,scaler)
  xtest  = scale_data(xtest,scaler)

  ytrain = one_hot_encoded(ytrain)
  ytest = one_hot_encoded(ytest)
  return xtrain,xtest,ytrain,ytest,testlabels

def splitData2(xtrain,ytrain,type):
  if(type=='master'):
    xtrain = xtrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
    ytrain = ytrain[(ytrain==1) | (ytrain==5)|(ytrain==7)]
  elif(type=='dynamic'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)]
  elif(type=='static'):
    xtrain = xtrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==4) | (ytrain==5)|(ytrain==6)]
  elif(type=='transition'):
    xtrain = xtrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
    ytrain = ytrain[(ytrain==7) | (ytrain==8)|(ytrain==9)|(ytrain==10)|(ytrain==11)|(ytrain==12)]
  elif(type=='primary'):
    xtrain = xtrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
    ytrain = ytrain[(ytrain==1) | (ytrain==2)|(ytrain==3)|(ytrain==4)|(ytrain==5)|(ytrain==6)]
  else:
    xtrain = xtrain
    ytrain = ytrain

  xtrain,xtest,ytrain,ytest = train_test_split(xtrain,ytrain,test_size = 0.3,random_state=42)

  return xtrain,xtest,ytrain,ytest


def resampleData(x,y):
  x = x.reshape(5773,768)
  df = pd.DataFrame(x)

  from imblearn.over_sampling import RandomOverSampler
  from collections import Counter
  from sklearn.datasets import make_classification
  from imblearn.over_sampling import SMOTE

  oversample = SMOTE()
  df, y = oversample.fit_resample(df, y)
  df=df.to_numpy()
  x=df.reshape(12228,128,6)
  x=df.reshape(12228,128,2,3)
  return x,y

# Data Preparation

## Loading data from files

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Preparing data, xtrain, ytrain
# Last six classes [7 to 12] has very less weightage in data since they are extra classes added
# , made from original six classes
# so, I took more overlapping in them to get slightly more data

xtrain,ytrain = prepare_data('/content/drive/MyDrive/HAPT Data Set/RawData',128,[0,0,0,0,0,0,0,0,0,0,0,0])

In [12]:
originalx = xtrain
originaly=ytrain

In [ ]:
#xtrain=originalx
#ytrain = originaly

In [13]:
ytrain.shape

(5773,)

In [14]:
xtrain.shape

(5773, 128, 2, 3)

In [ ]:
128*6

768

In [15]:
xtrain = xtrain.reshape(5773,768)


In [16]:
xtrain.shape

(5773, 768)

In [17]:
df = pd.DataFrame(xtrain)
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
3999,0.895833,-0.255556,0.377778,0.254731,-0.106901,-0.028405,1.0,-0.261111,0.434722,0.156382,...,-0.1,1.061989,-0.1799,-0.676533,0.5,-0.180556,-0.059722,1.05527,-0.127671,-0.6634


In [18]:
ytrain.shape

(5773,)

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
import tensorflow as tf
import datetime

# Clear any logs from previous runs
!rm -rf ./logs/

In [21]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
df, ytrain = oversample.fit_resample(df, ytrain)

In [ ]:
xtrain.shape

(5773, 768)

In [ ]:
df.shape

(12228, 768)

In [ ]:
ytrain.shape

(12228,)

In [22]:
df=df.to_numpy()
xtrain=df.reshape(12228,128,6)

In [ ]:
xtrain.shape

(12228, 128, 6)

In [23]:
xtrain=df.reshape(12228,128,2,3)

In [ ]:
xtrain.shape

(12228, 128, 2, 3)

**start SVM**

In [ ]:
X_train = pd.read_csv('/content/drive/MyDrive/newdata48features.csv', header=None)
xxx,Y_train = resampleData(originalx,originaly)

In [ ]:
X_train.head() # mean ,median , std , variance , root mean square , max, min

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0__median,0__mean,0__standard_deviation,0__variance,0__root_mean_square,0__maximum,0__absolute_maximum,0__minimum,1__median,1__mean,...,4__absolute_maximum,4__minimum,5__median,5__mean,5__standard_deviation,5__variance,5__root_mean_square,5__maximum,5__absolute_maximum,5__minimum
1,1.01944450285877,1.0193142942447153,0.0024762469750240646,6.131799081315831e-06,1.01931730204617,1.027777854158295,1.027777854158295,1.0125000434425,-0.1250000020616516,-0.12428385819818219,...,0.02138028293848038,-0.02138028293848038,0.003054326167330146,0.0024673228924712025,0.006133609869028267,3.762117002544095e-05,0.006611267070777992,0.01679879426956177,0.01679879426956177,-0.01404990069568157
2,1.01944450285877,1.0199002330079632,0.0033669412028527372,1.1336293063467436e-05,1.0199057905428133,1.02916674604155,1.02916674604155,1.009722259675992,-0.1263888939449057,-0.12505425802505304,...,0.03451388701796532,-0.03451388701796532,0.00366519158706069,0.00544290474636,0.009781032110216805,9.566858914109219e-05,0.011193471365891834,0.03634648025035858,0.03634648025035858,-0.01435533352196217
3,1.01944450285877,1.0201497995182354,0.002750501804412125,7.565260176074354e-06,1.0201535074278145,1.025000070391787,1.025000070391787,1.011111151559246,-0.1305555574387036,-0.13018663702089006,...,0.02626720629632473,-0.02626720629632473,0.0,0.0003436116994635086,0.009007026194748656,8.112652087288845e-05,0.009013578083807598,0.02718350291252136,0.02718350291252136,-0.02382374368607998
4,1.020833394742025,1.0207791411528349,0.0028706455352721085,8.24060578917769e-06,1.0207831775742136,1.027777854158295,1.027777854158295,1.013888935325754,-0.1263888939449057,-0.1276367229824334,...,0.03390302136540413,-0.03390302136540413,0.005345070967450738,0.006118197258047076,0.012574714847580413,0.00015812345349795927,0.013984126400541938,0.03359758853912354,0.03359758853912354,-0.02443460933864117


In [ ]:
Y_train.shape

(12228,)

In [ ]:
X_train=X_train.iloc[1: , :]


In [24]:
###########################################################################################
x1train,x1test,y1train,y1test=splitData2(X_train,Y_train,'dynamic')
x2train,x2test,y2train,y2test=splitData2(X_train,Y_train,'static')
x3train,x3test,y3train,y3test=splitData2(X_train,Y_train,'transition')

###########################################################################################
x1train.shape,y1train.shape,x3train.shape,y3train.shape

NameError: name 'X_train' is not defined

In [ ]:
X_train = np.concatenate([x1train,x2train, x3train], axis=0)
X_test = np.concatenate([x1test,x2test, x3test], axis=0)

Y_train = np.concatenate([y1train,y2train, y3train], axis=0)
Y_test = np.concatenate([y1test,y2test, y3test], axis=0)



In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size = 0.3,random_state=42)


In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((8557, 48), (3671, 48), (8557,), (3671,))

In [ ]:
X_test.shape,Y_test.shape

((3671, 48), (3671,))

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, accuracy_score


svc = LinearSVC(random_state=42, tol=1e-5, verbose=1, max_iter=10000)
svc.fit(X_train, Y_train)

svm_pred = svc.predict(X_test)
SVC_pred_test = accuracy_score(Y_test, svm_pred)*100
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(Y_test, svm_pred)*100))

[LibLinear]Accuracy score TEST: 90.902


In [ ]:
svm_pred.shape

(3671,)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100, 200]
scores=[]
for n_trees in tree_list:

    rf = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf.fit(x2train, y2train)
    rf_pred = rf.predict(x2test)
    scores.append(accuracy_score(y2test, rf_pred))
    print('Accuracy score of static activities TEST: {:.3f}'.format(accuracy_score(y2test, rf_pred)*100))

Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.534
Accuracy score of static activities TEST: 95.643
Accuracy score of static activities TEST: 95.752


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier

tree_list = [15, 50, 100, 200]
scores=[]
for n_trees in tree_list:

    rf2 = RandomForestClassifier(n_estimators=n_trees, random_state=40)
    rf2.fit(x3train, y3train)
    rf2_pred = rf2.predict(x3test)
    scores.append(accuracy_score(y3test, rf2_pred))
    print('Accuracy score of transition activities TEST: {:.3f}'.format(accuracy_score(y3test, rf2_pred)*100))

Accuracy score of transition activities TEST: 98.583
Accuracy score of transition activities TEST: 98.638
Accuracy score of transition activities TEST: 98.856
Accuracy score of transition activities TEST: 98.965


**end** **svm**

## Handling missing data and Normalizing data

In [25]:
xtrain,ytrain = remove_null(xtrain,ytrain)

In [26]:
xtrain.shape

(12228, 128, 2, 3)

In [ ]:
2449+2960+364

5773

In [ ]:
ytrain.shape

(12228,)

**Hier split and training**

**Dynamic**

In [27]:
dynamicxtrain,dynamicxtest,dynamicytrain,dynamicytest,dynamiclabels = splitData(xtrain,ytrain,'dynamic')

In [28]:
dynamicxtrain.shape,dynamicytrain.shape

((2139, 128, 2, 3), (2139, 3))

In [ ]:
####################################################################################################################################

In [29]:
dynamicxtrain.shape,dynamicxtest.shape,dynamicytrain.shape,dynamicytest.shape,dynamiclabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [30]:
#xtrain,ytrain = resampleData(originalx,originaly)
staticxtrain,staticxtest,staticytrain,staticytest,staticlabels = splitData(xtrain,ytrain,'static')
staticxtrain.shape,staticxtest.shape,staticytrain.shape,staticytest.shape,staticlabels.shape

((2139, 128, 2, 3), (918, 128, 2, 3), (2139, 3), (918, 3), (918,))

In [31]:
dynamicxtrain = dynamicxtrain.reshape(2139,128,6)
dynamicxtest = dynamicxtest.reshape(918,128,6)

In [32]:
staticxtrain = staticxtrain.reshape(2139,128,6)
staticxtest = staticxtest.reshape(918,128,6)

**start** **transformers**

In [33]:
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

from matplotlib import pyplot
import matplotlib.pyplot as plt
import keras
import zipfile
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [34]:
%cd sample_data

/content/sample_data


In [35]:
import model

learningRate = 3e-4

optimizer = tf.keras.optimizers.Adam(learningRate)
optimizer2 = tf.keras.optimizers.Adam(learningRate)

dynamic_classifier = model.HART((128,6),3)
static_classifier = model.HART((128,6),3)

dynamic_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)
static_classifier.compile(
    optimizer=optimizer2,
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"],
)

In [36]:
from sklearn.utils import class_weight

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(dynamiclabels),
                                                 y = dynamiclabels.ravel())
class_weights = {j : temp_weights[j] for j in range(len(temp_weights))}

temp_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(staticlabels),
                                                 y = staticlabels.ravel())
class_weights2 = {j : temp_weights[j] for j in range(len(temp_weights))}

In [37]:
np.unique(staticlabels)


array([4, 5, 6], dtype=uint32)

In [38]:
checkpoint_filepath = "bestValcheckpoint.h5"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

history0 = dynamic_classifier.fit(
    x=dynamicxtrain,
    y=dynamicytrain,
  # validation_data = (dynamicxtest,dynamicytest),
    batch_size=256,
    epochs=100,
    verbose=1,
    class_weight=class_weights,
    callbacks=[checkpoint_callback],
)



Epoch 1/100
9/9 [==============================] - ETA: 0s - loss: 1.0378 - accuracy: 0.4642

9/9 [==============================] - 62s 146ms/step - loss: 1.0378 - accuracy: 0.4642
Epoch 2/100
9/9 [==============================] - ETA: 0s - loss: 0.7879 - accuracy: 0.6994

9/9 [==============================] - 1s 92ms/step - loss: 0.7879 - accuracy: 0.6994
Epoch 3/100
9/9 [==============================] - ETA: 0s - loss: 0.7440 - accuracy: 0.7279

9/9 [==============================] - 1s 87ms/step - loss: 0.7440 - accuracy: 0.7279
Epoch 4/100
9/9 [==============================] - ETA: 0s - loss: 0.6921 - accuracy: 0.7695

9/9 [==============================] - 1s 91ms/step - loss: 0.6921 - accuracy: 0.7695
Epoch 5/100
8/9 [=========================>....] - ETA: 0s - loss: 0.6149 - accuracy: 0.8218

...

9/9 [==============================] - 1s 63ms/step - loss: 0.2949 - accuracy: 0.9995
Epoch 97/100
8/9 [=========================>....] - ETA: 0s - loss: 0.2936 - accuracy: 1.0000

9/9 [==============================] - 0s 49ms/step - loss: 0.2936 - accuracy: 1.0000
Epoch 98/100
9/9 [==============================] - ETA: 0s - loss: 0.2936 - accuracy: 1.0000

9/9 [==============================] - 0s 49ms/step - loss: 0.2936 - accuracy: 1.0000
Epoch 99/100
9/9 [==============================] - ETA: 0s - loss: 0.2940 - accuracy: 0.9995

9/9 [==============================] - 0s 50ms/step - loss: 0.2940 - accuracy: 0.9995
Epoch 100/100
8/9 [=========================>....] - ETA: 0s - loss: 0.2958 - accuracy: 0.9985

9/9 [==============================] - 0s 49ms/step - loss: 0.2956 - accuracy: 0.9986


In [ ]:
transscore = transition_classifier.evaluate(transitionxtest, transitionytest)
print("transition transformer model Accuracy: ", transscore[1])

58/58 [==============================] - 3s 22ms/step - loss: 0.4264 - accuracy: 0.9995
transition transformer model Accuracy:  0.9994550347328186


In [ ]:
yexpect = np.concatenate([dynamiclabels,staticlabels, transitionlabels], axis=0)

yexpect.shape

(3671,)

In [ ]:
merge_arr = np.concatenate([dynamicxtest, staticxtest, transitionxtest], axis=0)


In [ ]:
merge_arr.shape

(3671, 128, 6)

In [ ]:
X_test.shape,Y_test.shape

((3671, 48), (3671,))

In [ ]:
print('Accuracy score TEST: {:.3f}'.format(accuracy_score(yexpect, scores)*100))
# (static rf)(dynamic transformer)(transition rf) :  98.120%
# (static rf)(dynamic transformer)(transition transformer):  98.611 %

Accuracy score TEST: 98.611
